## IMPORTS

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import sys
import zipfile
import os
import matplotlib as ml
import matplotlib.pyplot as plt

In [ ]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

from sklearn.model_selection import cross_val_score

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, RandomTreesEmbedding
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

## GET DATA

In [ ]:
def load_train():
    return pd.read_csv('datasets/train.csv')

In [ ]:
def load_test():
    return pd.read_csv('datasets/test.csv')

In [ ]:
train = load_train()
test = load_test()
df = load_train()

## INICIAL DATA ANALYSIS

##### TRANSFORMAÇÕES TEMPORÁRIAS

In [ ]:
def encontrar_pronome_tratamento(texto):
    if ', ' in texto:
        texto = texto.split(', ')[1]
        return texto.split('.')[0]

In [ ]:
# A extração estratificada por categoria não funciona se as categorias possuirem apenas um elemento. 
def ajuste(cat):
    s = df['Tratamento'].value_counts()
    if s[cat] <= 1:
        return 'another'
    else:
        return cat

In [ ]:
# Transformando o sexo em valor binário
df['SexBin'] = df['Sex'].map({'male': 0, 'female': 1})

In [ ]:
df['Tratamento'] = df['Name'].apply(lambda x: encontrar_pronome_tratamento(x))
df['Tratamento'] = df['Tratamento'].apply(lambda x: ajuste(x))
df['TratamentoCat'], mapping = df['Tratamento'].factorize()

In [ ]:
df.corr()['Survived']

##### SEX

In [ ]:
df['Sex'].value_counts() 

In [ ]:
# Percentual de sobreviventes por sexo.
sobreviventes = df[df['Survived']==1]
sobreviventes['Sex'].value_counts() / df['Sex'].value_counts()

In [ ]:
sobreviventes['Tratamento'].value_counts()

In [ ]:
sns.catplot(data=df, kind='swarm', x="Sex", y='Age', hue='Survived')

In [ ]:
sns.catplot(data=df[df['Survived']==1], kind='bar', x="Sex", y='Age', hue='Tratamento')

In [ ]:
sns.catplot(data=df, kind='bar', x="Pclass", y='Age', hue='Survived')

In [ ]:
df['Pclass'].value_counts() / len(train)

## SETS

In [ ]:
labels = ['Age', 'Fare', 'Pclass', 'Sex', 'Embarked', 'TratamentoCat']

In [ ]:
from sklearn.model_selection import train_test_split

treino, validacao = train_test_split(df, test_size=0.3, random_state=42, shuffle=True)

X_treino = treino.drop("Survived", axis=1)
y_treino = treino['Survived']

X_valid = validacao.drop("Survived", axis=1)
y_valid = validacao['Survived']

In [ ]:
# AMOSTRA ESTRATIFICADA POR CATEGORIA
from sklearn.model_selection import StratifiedShuffleSplit

strat_train_set, strat_valid_set = [], []

sp = StratifiedShuffleSplit(n_splits=1, test_size=0.3, random_state=42)
for train_index, valid_index in sp.split(df, df['TratamentoCat']):
    strat_train_set = df.loc[train_index]
    strat_valid_set = df.loc[valid_index]


X_treino = strat_train_set.drop("Survived", axis=1)
y_treino = strat_train_set['Survived']

X_valid = strat_valid_set.drop("Survived", axis=1)
y_valid = strat_valid_set['Survived']

## TRASFORMING DATA

#### Pipelines

In [ ]:
pipe_fill_scaler = Pipeline([
    ('imp', SimpleImputer(strategy='median')), 
    ('scale', StandardScaler())
])

In [ ]:
pipe_cat_attribs = Pipeline([
    ('imp', SimpleImputer(strategy='most_frequent'))
    #('num', OneHotEncoder()) 
])

In [ ]:
transforms = ColumnTransformer([
    ('fill_scaler', pipe_fill_scaler, ['Age', 'Fare']), 
    ('cat_attribs', pipe_cat_attribs, ['Pclass', 'Sex', 'Embarked', 'TratamentoCat'])
])

## UTIL

In [ ]:
def display_scores(y_valid, y_predict):
    print("precision_score: {}\n".format(precision_score(y_valid, y_predict)))
    print("recall_score: {}\n".format(recall_score(y_valid, y_predict)))
    print("confusion_matrix: \n{}\n".format(confusion_matrix(y_valid, y_predict)))
    print("classification_report: \n{}".format(classification_report(y_valid, y_predict)))


## CROS-VAL-SCORE

In [ ]:
from sklearn.metrics import precision_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, roc_curve
from sklearn.model_selection import cross_val_predict

In [ ]:
models = [SVC()]#[DecisionTreeClassifier(), RandomForestClassifier(), SGDClassifier(), KNeighborsClassifier(), SVC(), LogisticRegression()]

for model in models:
  pipe = make_pipeline(transforms, model)
  val_scores = cross_val_score(pipe, X_treino[labels], y_treino, cv=3)
  nome_modelo = type(model).__name__ # somente para exibição
  print('Modelo: {} | Média: {:.2} | Desvio: {:.2}'.format(nome_modelo, np.mean(val_scores), np.std(val_scores)))

## SVC

In [ ]:
model = SVC()
model.fit(transforms.fit_transform(X_treino[labels]), y_treino)

In [ ]:
y_predict = model.predict(transforms.fit_transform(X_valid[labels]))

In [ ]:
display_scores(y_valid, y_predict)

## SUBMISSÃO KAGGLE